# RFM analysis 

<img src="https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true">

> Author: Serge Retkowsky Microsoft<br>
> Date: 03-Sept-2020

## Description
RFM Analysis is a marketing technique used to quantitatively determine which customers are the best ones by examining their shopping behaviour – how recently a customer has purchased (recency), how often they purchase (frequency), and how much the customer spends (monetary). RFM analysis is based on an extension of Pareto’s principle which says that “80% of your business comes from 20% of your customers.“

## Objectives
Customers who have purchased more recently, more frequently, and have spent more money, are likelier to buy again. But those who haven’t, are less valuable for the company and therefore, likely to churn. RFM stands for:

**Recency** – How recently did the customer purchase? Recency is the most important predictor of who is more likely to show loyalty towards your brand. Customers who have purchased recently from you are more likely to purchase again from you compared to those who did not purchase recently.

**Frequency** – How often do they purchase? The second most important factor is how frequently these customers purchase from you. The higher the frequency, the higher is the chances of such customers making a repeat purchase.

**Monetary** – How much money do they spend (average basket value)?

## Steps

1. We will connect to the Azure ML workspace in order to save our ML results into an experiment.
2. We will take the customer transactions and build a RFM clustering. 3 indicators (Recency, Frequency and Monetary) and a 
global RFM score will be calculated for each customer.
3. We are able to select the top RFM scores because this cluster is all customers with a low recency, high frequency and a high
Monetary. 

<img src="http://www.wiseguysmarketing.com/wp-content/uploads/2016/03/RFM.png">

## 0. Settings

In [ ]:
import sys
print('You are using Python', sys.version)

In [2]:
import datetime
dateheure = datetime.datetime.now()
print('Today is', dateheure)

Today is 2020-09-03 10:31:19.795417


In [3]:
import azureml.core
print("You are using Azure ML", azureml.core.VERSION)

You are using Azure ML 1.13.0


In [4]:
import pandas as pd
import logging
import os
import random

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

In [5]:
import os
subscription_id = os.environ.get("SUBSCRIPTION_ID", "70b8f39e-8863-49f7-b6ba-34a80799550c")
resource_group = os.environ.get("RESOURCE_GROUP", "azuremlsynapse-rg")
workspace_name = os.environ.get("WORKSPACE_NAME", "azuremlsynapse")

from azureml.core import Workspace
try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("OK")
except:
   print("Error: Workspace not found")

OK


In [6]:
from azureml.core import Workspace

try:
   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
   ws.write_config()
   print("Workspace is available : OK")
except:
   print("No Workspace")

Workspace is available : OK


## 1. Azure ML experimentation

In [7]:
ws = Workspace.from_config()

experiment = Experiment(workspace=ws, name='RFMAnalysis')

output = {}
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
Workspace,azuremlsynapse
Resource Group,azuremlsynapse-rg
Location,westeurope
Experiment Name,RFMAnalysis


## 2. Data Access

In [8]:
from azureml.core import Workspace, Dataset

subscription_id = '70b8f39e-8863-49f7-b6ba-34a80799550c'
resource_group = 'azuremlsynapse-rg'
workspace_name = 'azuremlsynapse'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Transactions')
ventes=dataset.to_pandas_dataframe()

In [9]:
ventes.head()

,DateAchat,CodeClient,NumeroCommande,NumeroLigneProduit,IDMagasin,PrixVenteTTCHorsRemise,PrixVenteTTCAvecRemise,Quantite,Montant,Heure,CodeVendeur,DateCreationArticle,LabelProduit,LabelSousFamille,LabelFamilleProduit,LabelUnivers,CodeUnivers,CodeFamille,CodeSousFamille,CodeProduit
0,2019-09-18,3136178,12238,1,10001,38.986667,38.986667,1,38.986667,1807.0,28,2014-08-25,CHANEL ECLAT EXPRESS 75 ML,MASQUE,VISAGE,SOINS,3,7,30,4
1,2019-12-18,2665443,1418,3,10000,0.000000,0.000000,1,0.000000,1533.0,10,2007-02-01,PETIT POT ECHT,FOURNITURES,AUTRES,CONSEILLERS SOINS,10,100,160,284
2,2019-10-17,3502093,14510,1,10001,5.053333,5.053333,1,5.053333,1118.0,15,2018-08-19,MAVALA LIMES A ONGLES X8,SOINS ONGLES,LES ONGLES,MAQUILLAGE,2,4,24,320
3,2018-08-21,4651928,16477,5,10000,6.026667,6.026667,1,6.026667,1543.0,13,2018-08-19,MAVALA MINI VAO SUPER BASE,VERNIS,LES ONGLES,MAQUILLAGE,2,4,22,322
4,2020-05-18,2304176,3489,6,10000,8.133333,8.133333,1,8.133333,1553.0,5,2018-08-19,MAVALA DISSOLV DISQUE POT,DISSOLVANT,LES ONGLES,MAQUILLAGE,2,4,23,323


In [10]:
import time
RFMref = time.strftime("%Y-%m-%d")
print('RFM Reference Date is', RFMref)

RFM Reference Date is 2020-09-03


In [11]:
run = experiment.start_logging(snapshot_directory=None )

In [12]:
run.log('RFM ref', RFMref)

In [13]:
ventes['DateAchat'] = pd.to_datetime(ventes['DateAchat'])

In [14]:
import datetime
NOW = datetime.datetime.now()

In [15]:
TableRFM = ventes.groupby('CodeClient').agg({'DateAchat': lambda x: (NOW - x.max()).days, # Recency
                                        'NumeroCommande': lambda x: len(x),      # Frequency
                                        'Montant': lambda x: x.sum()}) # Monetary Value

TableRFM['DateAchat'] = TableRFM['DateAchat'].astype(int)
TableRFM.rename(columns={'DateAchat': 'Récence', 
                         'NumeroCommande': 'Fréquence', 
                         'Montant': 'Montant'}, inplace=True)

## 3. RFM analysis

In [16]:
TableRFM.head(20)

,Récence,Fréquence,Montant
CodeClient,,,
126993,169,1,26.000000
214674,138,3,116.800000
318696,108,2,73.733334
438711,322,3,102.133334
495501,434,3,108.933333
918238,596,1,80.666667
1090522,287,1,31.466667
1315018,351,1,20.000000
1419361,260,1,39.333333


In [17]:
# Some details for a customer ID
Client3136178=ventes[ventes['CodeClient']==3136178]
Client3136178

,DateAchat,CodeClient,NumeroCommande,NumeroLigneProduit,IDMagasin,PrixVenteTTCHorsRemise,PrixVenteTTCAvecRemise,Quantite,Montant,Heure,CodeVendeur,DateCreationArticle,LabelProduit,LabelSousFamille,LabelFamilleProduit,LabelUnivers,CodeUnivers,CodeFamille,CodeSousFamille,CodeProduit
0,2019-09-18,3136178,12238,1,10001,38.986667,38.986667,1,38.986667,1807.0,28,2014-08-25,CHANEL ECLAT EXPRESS 75 ML,MASQUE,VISAGE,SOINS,3,7,30,4
653,2019-09-18,3136178,12238,2,10001,63.066667,63.066667,1,63.066667,1807.0,28,2010-11-28,LAUDER EYZONE REPAIR GEL15,YEUX,VISAGE,SOINS,3,7,1009,32311
2509,2019-09-18,3136178,12238,3,10001,62.373333,62.373333,1,62.373333,1807.0,28,2022-01-10,LAUDER HYDRACOMPLETE PN 50,HYDRATANT,VISAGE,SOINS,3,7,31,97599
2571,2019-09-18,3136178,12238,4,10001,50.986667,50.986667,1,50.986667,1807.0,28,2022-02-02,GUERLAIN OAP DIVI.4COUL280,OMBRES PAUPIERES,LES YEUX,MAQUILLAGE,2,2,16,98707


In [18]:
quantiles = TableRFM.quantile(q=[0.25,0.5,0.75])

In [19]:
quantiles

,Récence,Fréquence,Montant
0.25,260.0,1.0,27.826667
0.50,351.0,1.0,53.066667
0.75,434.0,2.0,87.700000


In [20]:
quantiles = quantiles.to_dict()

In [21]:
quantiles

{'Récence': {0.25: 260.0, 0.5: 351.0, 0.75: 434.0},
 'Fréquence': {0.25: 1.0, 0.5: 1.0, 0.75: 2.0},
 'Montant': {0.25: 27.8266665, 0.5: 53.066666999999995, 0.75: 87.7}}

In [22]:
SegmentationRFM = TableRFM

In [23]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

### Let's compute the Recency, Frequency and Monetary indicators.

In [24]:
SegmentationRFM['R_Quartile'] = SegmentationRFM['Récence'].apply(RClass, args=('Récence',quantiles,))
SegmentationRFM['F_Quartile'] = SegmentationRFM['Fréquence'].apply(FMClass, args=('Fréquence',quantiles,))
SegmentationRFM['M_Quartile'] = SegmentationRFM['Montant'].apply(FMClass, args=('Montant',quantiles,))

### Let's compute the global RFMClass indicator for each customer

In [25]:
SegmentationRFM['RFMClass'] = SegmentationRFM.R_Quartile.map(str) \
                            + SegmentationRFM.F_Quartile.map(str) \
                            + SegmentationRFM.M_Quartile.map(str)

In [26]:
SegmentationRFM.head()

,Récence,Fréquence,Montant,R_Quartile,F_Quartile,M_Quartile,RFMClass
CodeClient,,,,,,,
126993,169,1,26.000000,1,4,4,144
214674,138,3,116.800000,1,1,1,111
318696,108,2,73.733334,1,2,2,122
438711,322,3,102.133334,2,1,1,211
495501,434,3,108.933333,3,1,1,311


In [27]:
SegmentationRFM.describe()

,Récence,Fréquence,Montant,R_Quartile,F_Quartile,M_Quartile
count,2255.000000,2255.000000,2255.000000,2255.000000,2255.000000,2255.000000
mean,365.074501,1.956098,69.354968,2.313525,2.794235,2.500222
std,175.898913,1.454829,65.745289,1.102417,1.288218,1.118480
min,25.000000,1.000000,0.000000,1.000000,1.000000,1.000000
25%,260.000000,1.000000,27.826667,1.000000,2.000000,1.500000
50%,351.000000,1.000000,53.066667,2.000000,4.000000,3.000000
75%,434.000000,2.000000,87.700000,3.000000,4.000000,3.500000
max,747.000000,22.000000,804.533334,4.000000,4.000000,4.000000


In [28]:
SegmentationRFM['RFMClass'] = SegmentationRFM['RFMClass'].astype(int)

### List of the top RFM customers

In [29]:
SegmentationRFM.sort_values(by=['RFMClass'], ascending=[False])

,Récence,Fréquence,Montant,R_Quartile,F_Quartile,M_Quartile,RFMClass
CodeClient,,,,,,,
2273695,744,1,22.133333,4,4,4,444
2248528,747,1,11.600000,4,4,4,444
4215108,744,1,13.333333,4,4,4,444
2255804,747,1,26.586667,4,4,4,444
4209466,688,1,26.000000,4,4,4,444
2874132,721,1,27.813333,4,4,4,444
2874172,688,1,7.866667,4,4,4,444
2874184,623,1,13.200000,4,4,4,444
4139781,688,1,12.266667,4,4,4,444


## 4. Let's choose the top RFM customers

In [30]:
seuil=400

In [31]:
#Let's log the value into the Azure ML experiment
run.log("RFM Class Seuil", seuil)
HighRFM=SegmentationRFM[SegmentationRFM['RFMClass'].gt(seuil)]

In [32]:
HighRFM

,Récence,Fréquence,Montant,R_Quartile,F_Quartile,M_Quartile,RFMClass
CodeClient,,,,,,,
918238,596,1,80.666667,4,4,2,442
1470868,744,1,5.466667,4,4,4,444
1560817,500,1,59.466667,4,4,2,442
1615577,623,2,42.800000,4,2,3,423
1747156,500,2,126.933334,4,2,1,421
1794794,623,1,69.200000,4,4,2,442
1796833,623,2,64.266667,4,2,2,422
1797430,747,1,0.666667,4,4,4,444
1894016,500,2,77.733334,4,2,2,422


## 5. Exportation of results

### Let's export the top RFM customers into a CSV and Excel file

In [33]:
HighRFM.to_csv('HighRFM.csv', sep=',')

In [34]:
HighRFM.to_excel('HighRFM.xlsx')

In [35]:
%ls HighRFM*.* -l

-rwxrwxrwx 1 root root 15162 Sep  3 10:31 HighRFM.csv*
-rwxrwxrwx 1 root root 21318 Sep  3 10:31 HighRFM.xlsx*


### Let's persist the top RFM file into an Azure ML experiment

In [36]:
file_namecsv = 'HighRFM.csv'
run.upload_file(name = file_namecsv, path_or_stream = file_namecsv)

In [37]:
file_namexls = 'HighRFM.xlsx'
run.upload_file(name = file_namexls, path_or_stream = file_namexls)

In [38]:
experiment

Name,Workspace,Report Page,Docs Page
RFMAnalysis,azuremlsynapse,Link to Azure Machine Learning studio,Link to Documentation


### Saving results into the Azure ML experimentation:
<img src='https://github.com/retkowsky/images/blob/master/rfm1.jpg?raw=true'>
<img src='https://github.com/retkowsky/images/blob/master/rfm2.jpg?raw=true'>

In [39]:
run.complete()

> End of notebook